In [1]:
import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
import gensim
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
import sklearn.pipeline
import re
from sklearn import svm
from sklearn import *
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
from scipy import sparse
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import LeaveOneOut,KFold,train_test_split
import simplejson
import pprint


# Custom imports
from mr_generic_scripts import *
from mr_cls_BILSTM import *
from mr_generic_scripts import load_combined_data

In [2]:
# # Path to xlsx files folder
# Define paths to the available CSV files
path_to_original_data = 'Data/original_dataset.csv'  # Path to the original data
path_to_augmented_data = 'Data/augmented_dataset.csv'  # Path to the augmented data

# Update any other references to these paths in your code as needed

# # Original MIND-CA corpus
# path_to_raw_files = f_path + 'Data/raw_relabeled/'
# # MIND-CA + human augment
# path_to_plus_files = f_path + 'Data/raw_plus/'
# # UK-MIND-20
# path_to_wp1 = f_path + 'Data/wp1/'

# # Augmented data
# # augmentations, 125 examples per QA pair
# path_to_aug = f_path + 'Data/aug_data/all/'
# # augmentations, 500 examples per QA pair
# path_to_aug_hq_os = f_path + 'Data/aug_data_os/all/'
# # augmentation, no sampling - 1500 total examples per question
# path_to_aug_joint = f_path + 'Data/aug_data_joint/all/'

# # Merged xlsx files with multiple augmentations
# path_to_set_files = f_path + 'Data/aug_data/sets/'

In [3]:
# List of augmentations by category
hq_data = ['reord','phrase','dict']
lq_data = ['wordnet','ppdb','glove','fasttext']
set_data = ['ab_lq','ab_hq','all_lq','all_hq','all_aug']

In [4]:
# General config of the training run (!)

# List of data to use for training
# Adjust the train_sets based on available data
train_sets = ['original', 'augmented']  # This includes only the files you have

# Alias paths to your data
aug_path = '../Data/augmented_dataset.csv'
orig_path = '../Data/original_dataset.csv'

# Training parameters
# Number of folds for k-fold cross-validation
n_k_fold = 3

# Only answers (False) or questions + answers (True)
mind_qa = True

# Function to load train data based on train_sets selection
def load_train_data(train_set):
    if train_set == 'original':
        return pd.read_csv(orig_path)
    elif train_set == 'augmented':
        return pd.read_csv(aug_path)
    else:
        raise ValueError(f"Unknown train set: {train_set}")

# Example of loading and using data
for train_set in train_sets:
    train_data = load_train_data(train_set)
    # Proceed with training logic for each dataset in train_sets
    print(f"Loaded {train_set} dataset with shape:", train_data.shape)



# # General config of the training run (!)

# # List of data to use for training

# # All possible train sets
# # train_sets = ['orig','plus','reord','phrase','dict','wordnet','ppdb','glove','fasttext','ab_lq','ab_hq','all_lq','all_hq']

# # Selective train set
# #train_sets = ['reord','phrase','dict']
# train_sets = ['wordnet','ppdb','glove','fasttext']
# #train_sets = ['wp1']

# # Alias path to aug data (either 125 or 500 examples or the 1500 joint)
# aug_path = path_to_aug_joint

# # Training parameters
# # Number of folds for k-fold cross validation
# n_k_fold = 10

# # Only answers (False) or questions + answers (True)
# mind_qa = True

Loaded original dataset with shape: (100, 7)
Loaded augmented dataset with shape: (20, 7)


In [5]:
# Define paths directly to the CSV files in your current folder structure
path_to_raw_files = '../Data/original_dataset.csv'  # Replace with your actual path if needed
path_to_plus_files = '../Data/augmented_dataset.csv'  # Replace with your actual path if needed

# If there are additional augmented data paths, add them here
path_to_aug = '../Data/augmented_dataset.csv'  # Example path for augmented data

# Initialize datasets dictionary
datasets = {}

# Load combined data
combined_data = load_combined_data()

# Check if we load only answers or questions plus answers
if mind_qa:
    # Load the base dataset (augmented data)
    datasets['plus'] = pd.read_csv(path_to_plus_files)  # assuming it’s CSV format
    
    # Load the original dataset if needed
    if 'orig' in train_sets:
        datasets['orig'] = pd.read_csv(path_to_raw_files)

    # Load augmented data
    for at_set in train_sets:
        if at_set in ['orig', 'plus']:
            continue
        datasets[at_set] = pd.read_csv(path_to_aug)

else:
    # Only answers
    datasets['plus'] = pd.read_csv(path_to_plus_files)
    
    if 'orig' in train_sets:
        datasets['orig'] = pd.read_csv(path_to_raw_files)
    
    # Load augmented data
    for at_set in train_sets:
        if at_set in ['orig', 'plus']:
            continue
        datasets[at_set] = pd.read_csv(path_to_aug)

# # Get the datasets in dataframes
# combined_data = load_combined_data()

# # Check if we load only answers or questions plus answers
# if mind_qa:
#     # Always load MIND-CA + human aug, this is the base set
#     datasets['plus'] = mr_get_qa_data(path_to_plus_files)

#     # Always load UK-MIND-20, we need it for testing
#     datasets['wp1'] = mr_get_qa_data(path_to_wp1)

#     # If comparison is needed, load MIND-CA without any aug
#     if 'orig' in train_sets:
#         datasets['orig'] = mr_get_qa_data(path_to_raw_files)

#     # Load augmented data
#     for at_set in train_sets:
#         if at_set in ['orig','plus','wp1']:
#             continue
#         path_to_aug = aug_path + at_set + "/"
#         datasets[at_set] = mr_get_qa_data(path_to_aug)

    
# # Only the answer
# else:
#     # Always load MIND-CA + human aug, this is the base set
#     datasets['plus'] = mr_get_data(path_to_plus_files)

#     # Always load UK-MIND-20, we need it for testing
#     datasets['wp1'] = mr_get_data(path_to_wp1)

#     # If comparison is needed, load MIND-CA without any aug
#     if 'orig' in train_sets:
#         datasets['orig'] = mr_get_data(path_to_raw_files)

#     # Load augmented data
#     aug_dataset = {}
#     for at_set in train_sets:
#         if at_set in ['orig','plus','wp1']:
#             continue
#         path_to_aug = aug_path + at_set + "/"
#         datasets[at_set] = mr_get_data(path_to_aug)

In [6]:
# Sanity check
for d_id in train_sets:
    print(f"Dataset '{d_id}' length:", len(datasets[d_id]))

    # Skip the base datasets that do not have an 'Aug_ID' column
    if d_id in ['orig', 'plus', 'wp1']:
        continue

    # Check if 'Aug_ID' exists before dropping it
    if 'Aug_ID' in datasets[d_id].columns:
        datasets[d_id].drop(["Aug_ID"], axis=1, inplace=True)
    else:
        print(f"'Aug_ID' column not found in dataset '{d_id}', skipping drop.")

# # Sanity check
# for d_id in train_sets:
#     print(len(datasets[d_id][-1][1]))
#     if at_set in ['orig','plus','wp1']:
#          continue
#     # Augmented datasets have additional column that needs to be dropped
#     datasets[d_id][-1][1].drop(["Aug_ID"],axis=1,inplace=True)

Dataset 'original' length: 20
'Aug_ID' column not found in dataset 'original', skipping drop.
Dataset 'augmented' length: 20
'Aug_ID' column not found in dataset 'augmented', skipping drop.


In [7]:
from itertools import chain  # For flattening nested lists

def flatten(lst):
    """Helper function to flatten a nested list."""
    return list(chain.from_iterable(lst)) if isinstance(lst[0], list) else lst

def mr_proc_results(raw_results):
    pr_results = []

    for entry in raw_results:
        try:
            # Adapted for a nested list structure as seen in the screenshot
            acc_score = entry[0][0]
            f1_score = entry[1][0]
            acc_score_wp1 = entry[2][0]
            f1_score_wp1 = entry[3][0]

            # Assuming the second element in each entry list is a list of scores for questions
            qf_s = entry[1][1] if len(entry[1]) > 1 else []
            qf_s_wp1 = entry[3][1] if len(entry[3]) > 1 else []

            # Flatten and calculate the mean for question F1 scores if available
            avg_qf_s = round(sum(qf_s) / len(qf_s), 2) if qf_s else 0
            avg_qf_s_wp1 = round(sum(qf_s_wp1) / len(qf_s_wp1), 2) if qf_s_wp1 else 0

            # Append processed result in the expected format
            pr_entry = [
                [acc_score, f1_score, avg_qf_s],
                [acc_score_wp1, f1_score_wp1, avg_qf_s_wp1]
            ]
            pr_results.append(pr_entry)

        except (TypeError, IndexError) as e:
            print(f"Error processing entry: {entry}, error: {e}")

    # Convert the processed results to a numpy array for further processing
    pr_arr = np.array(pr_results)
    print("Processed Results Array:", pr_arr)
    return pr_arr


# def mr_proc_results(raw_results):
#   # Process the results from the 10 runs
#   # result format: [acc, acc per q, acc per age], [f1, f1 per q, f1 per age], [acc, acc per q, acc per age] (for wp1), [f1, f1 per q, f1 per age] (for wp1)
#   # Ignore ages as they seem to be mostly consistent with global average
#   # Ignore accs per question and age as averaging them seems to be consistent with global average
#   # Report global acc, global macro f1, average of macro f1 per question; same for wp1
#   pr_results = [[[acc_score, f1_score,round(sum(qf_s)/11,2)],[acc_score_wp1, f1_score_wp1,round(sum(qf_s_wp1)/11,2)]] 
#                 for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s],
#                      [acc_score_wp1, qa_s_wp1, aa_s_wp1], [f1_score_wp1, qf_s_wp1, af_s_wp1]) in raw_results]

#   # Throw the list in an np array
#   pr_arr = np.array(pr_results)

#   # Print the results
#   pp = pprint.PrettyPrinter(indent=4)

#   pp.pprint(pr_results)
#   pp.pprint(np.mean(pr_arr,axis=0))

In [8]:
# Initialize the classifier
if mind_qa:
  # ages 8 to 13, removing outliers; vocabulary of 1000; max len 35
  bl_cls = MR_bilstm(text_cols,[7,8,9,10,11,12,13,14],1000,35)
else:
  # ages 8 to 13, removing outliers; vocabulary of 1000; max len 20
  bl_cls = MR_bilstm(text_cols,[7,8,9,10,11,12,13,14],1000,20)

# Configure eval parameters - eval by age and questions, do not return examples with errors (not fully implemented in current version)  
bl_cls.mr_set_eval_vars(True,True,False)

In [9]:
# Initialize the results variable 
results = {}

In [10]:
# Use the combined dataset for training and evaluation
train_df, test_df = train_test_split(combined_data, test_size=0.2)

# Train and evaluate the model directly, bypassing k-fold cross-validation
bl_cls.mr_train(train_df, test_df)  # Train the model on the train-test split

# Test the trained model on the test set
test_accuracy, test_f1 = bl_cls.mr_test(test_df)  # Evaluate on the test set
print(f"Test Accuracy: {test_accuracy}, Test F1 Score: {test_f1}")


# if train_df.empty or test_df.empty:
#     raise ValueError("Train or test DataFrame is empty. Please check data loading.")

# # Train and evaluate using k-fold cross-validation
# results['combined'] = bl_cls.mr_kfold_pre_split(train_df, test_df, 0.25, n_k_fold)

# # Save the results
# rs_path = 'Results/split_eval_joint/bl_qa_combined.txt'
# with open(rs_path, 'w') as op:
#     simplejson.dump(results['combined'], op)


# Run all train-test combos
# for at_set in train_sets:
#     print("Current train: " + str(at_set) + "\n")

#     if at_set in ['orig','plus','wp1']:
#         # For orig and plus we directly train and test using kfold validation
#         results[at_set] = bl_cls.mr_kfold_pre_split(datasets[at_set][-1][1],datasets['wp1'][-1][1],0.25,n_k_fold)
#     else:
#         # For augmented data we need to also provide the "plus" set for evaluation and organizing the split
#         results[at_set] = bl_cls.mr_kfold_aug_pre_split(datasets['plus'][-1][1],datasets[at_set][-1][1],datasets['wp1'][-1][1],0.25,n_k_fold)

#     # Save the results in a file
#     rs_path = 'Results/split_eval_joint/bl_qa_os_'
#     s_path = rs_path + at_set + '.txt'
#     with open(s_path,'w') as op:
#         simplejson.dump(results[at_set],op)

Train DataFrame size: 120
Validation DataFrame size: 24
Input shape: 35
Inferred batch size from train_dataset: 4


2024-11-09 20:43:34.583517: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 35, 64)         │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,499 (541.01 KB)

 Trainable params: 138,499 (541.01 KB)

 Non-trainable params: 0 (0.00 B)

None

 Training
Columns in test_df: Index(['ID', 'Question', 'Answer', 'Score', 'Age', 'Gender', 'Orig_ID'], dtype='object')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Test Accuracy: 0.375
Test Macro F1: 0.1818181818181818
Test Accuracy: 0.375, Test F1 Score: 0.1818181818181818


In [11]:
import simplejson
import os

# Initialize the results dictionary
results = {}

# Define the base path
base_path = "/Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results"  

# Define the filenames explicitly based on your directory structure
file_structure = {
    "Imbalanced": [
        "tr_qa_os_phrase.txt", "tr_qa_os_ppdb.txt", "tr_qa_os_reord.txt", "tr_qa_os_wordnet.txt"
    ],
    "Original": [
        "bl_a_dict.txt", "bl_a_fasttext.txt", "bl_a_glove.txt", "bl_a_orig.txt",
        "bl_a_phrase.txt", "bl_a_plus.txt", "bl_a_ppdb.txt", "bl_a_reord.txt",
        "bl_a_wordnet.txt", "bl_qa_dict.txt", "bl_qa_fasttext.txt", "bl_qa_glove.txt"
        # Add the rest of the filenames from 'Original' here
    ],
    "OverSampling": [
        "10f_tr_qa_os_glove.txt", "10f_tr_qa_os_ppdb.txt", "10f_tr_qa_os_wordnet.txt",
        "10f_tr_qa_os_dict.txt", "10f_tr_qa_os_fasttext.txt"
        # Add the rest of the filenames from 'OverSampling' here
    ]
}

# Load files into the results dictionary
for subfolder, filenames in file_structure.items():
    results[subfolder] = {}
    for filename in filenames:
        file_path = os.path.join(base_path, subfolder, filename)
        print(f"Attempting to load file: {file_path}")
        try:
            with open(file_path, "r") as inp:
                # Store content using filename without extension as key
                results[subfolder][filename.replace(".txt", "")] = simplejson.load(inp)
        except FileNotFoundError:
            print(f"File not found: {file_path}")

# Combine all results into a single entry under 'combined'
combined_data = []
for subfolder in results:
    for data_key in results[subfolder]:
        combined_data.append(results[subfolder][data_key])

# Store combined results
results['combined'] = combined_data

# Step 2: Reformat `results['combined']` if necessary
# Assuming that each entry in `results['combined']` needs to be structured as:
# [[acc_score, qa_s, aa_s], [f1_score, qf_s, af_s], [acc_score_wp1, qa_s_wp1, aa_s_wp1], [f1_score_wp1, qf_s_wp1, af_s_wp1]]
# Reformatted combined data storage
reformatted_combined = []

# Reformat each entry in combined results
for entry in results['combined']:
    try:
        # Access nested list elements based on observed structure
        reformatted_entry = [
            [entry[0][0], entry[0][1], entry[0][2]],  # Acc Score, Question Scores, Age Scores
            [entry[1][0], entry[1][1], entry[1][2]],  # F1 Score, Question F1 Scores, Age F1 Scores
            [entry[2][0], entry[2][1], entry[2][2]],  # Acc Score WP1, Question Scores WP1, Age Scores WP1
            [entry[3][0], entry[3][1], entry[3][2]]   # F1 Score WP1, Question F1 Scores WP1, Age F1 Scores WP1
        ]
        reformatted_combined.append(reformatted_entry)
    except (IndexError, TypeError) as e:
        print(f"Error processing entry: {entry}, error: {e}")

# Replace `results['combined']` with the reformatted version
results['combined'] = reformatted_combined

# Verify reformatted combined results
print("Reformatted combined results:", results['combined'][:2])  # Display first two entries to verify



Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Imbalanced/tr_qa_os_phrase.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Imbalanced/tr_qa_os_ppdb.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Imbalanced/tr_qa_os_reord.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Imbalanced/tr_qa_os_wordnet.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Original/bl_a_dict.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Original/bl_a_fasttext.txt
Attempting to load file: /Users/kaushikkarthikeyan/Documents/SNR3/570/augment-acl21-enhanced/Experiments/Results/Original/bl_a_glove.txt
Attempting to

In [12]:

# Visualize the results
print("Combined dataset results:")
print("Available keys in results:", results.keys())

# Inspect the structure of results['combined']
print("Sample of combined results:", results['combined'][:2])  # Print first two items for inspection


mr_proc_results(results['combined'])


# for at_set in train_sets:
#     print(at_set)
#     mr_proc_results(results[at_set])

Combined dataset results:
Available keys in results: dict_keys(['Imbalanced', 'Original', 'OverSampling', 'combined'])
Sample of combined results: [[[[0.97, [1.0, 0.97, 0.97, 0.96, 0.98, 0.96, 0.98, 0.97, 0.95, 0.98, 0.96], [0.94, 0.98, 0.97, 0.96, 0.99, 0.98]], [0.97, [0.99, 0.96, 0.96, 0.96, 0.98, 0.95, 0.97, 0.94, 0.95, 0.97, 0.96], [0.94, 0.98, 0.97, 0.96, 0.99, 0.97]], [0.9, [0.91, 0.84, 0.92, 0.87, 0.91, 0.88, 0.94, 0.95, 0.88, 0.92, 0.9], [0.93, 0.91, 0.9, 0.91, 0.88, 0.89]]], [[0.97, [0.97, 0.97, 0.99, 0.94, 0.97, 0.97, 0.98, 0.95, 0.99, 0.97, 0.97], [0.98, 0.96, 0.97, 0.97, 0.97, 0.98]], [0.97, [0.95, 0.94, 0.98, 0.93, 0.95, 0.96, 0.97, 0.92, 0.98, 0.96, 0.97], [0.98, 0.96, 0.97, 0.97, 0.97, 0.98]], [0.9, [0.91, 0.84, 0.92, 0.87, 0.93, 0.87, 0.94, 0.95, 0.88, 0.92, 0.89], [0.91, 0.91, 0.9, 0.92, 0.88, 0.9]]], [[0.97, [0.96, 0.96, 0.98, 0.93, 0.99, 0.96, 0.98, 0.99, 0.97, 0.98, 0.97], [0.97, 0.97, 0.98, 0.96, 0.95, 0.96]], [0.97, [0.95, 0.94, 0.97, 0.91, 0.99, 0.95, 0.97, 0.98,

array([], dtype=float64)